In [14]:
import pandas as pd
from sqlalchemy import create_engine, text
from dateutil.relativedelta import relativedelta

In [4]:
buildingPermits = pd.read_csv('BuildingPermitDimension.csv',low_memory=False)
demolition = pd.read_csv('DemolitionDimension.csv',low_memory=False)


In [17]:

# changing date types
buildingPermits['Issued_Date'] = pd.to_datetime(buildingPermits['ISSUED_DATE'])
buildingPermits['APPLICATION_DATE'] = pd.to_datetime(buildingPermits['APPLICATION_DATE'])

# changing int types
buildingPermits['EST_CONST_COST'] = buildingPermits['EST_CONST_COST'].str.replace(',', '')  # Remove commas
buildingPermits['EST_CONST_COST'] = pd.to_numeric(buildingPermits['EST_CONST_COST'], errors='coerce')
buildingPermits['EST_CONST_COST'] = buildingPermits['EST_CONST_COST'].fillna(0).astype(int)
buildingPermits['GEO_ID'] = pd.to_numeric(buildingPermits['GEO_ID'], errors='coerce').astype('Int64')

# changing string types
buildingPermits['PERMIT_NUM'] = buildingPermits['PERMIT_NUM'].astype(str)
buildingPermits['PERMIT_TYPE'] = buildingPermits['PERMIT_TYPE'].astype(str)
buildingPermits['WORK'] = buildingPermits['WORK'].astype(str)
buildingPermits['STREET_NAME'] = buildingPermits['STREET_NAME'].astype(str)
buildingPermits['STREET_TYPE'] = buildingPermits['STREET_TYPE'].astype(str)
buildingPermits['STREET_NUM'] = buildingPermits['STREET_NUM'].astype(str)
buildingPermits['STREET_DIRECTION'] = buildingPermits['STREET_DIRECTION'].astype(str)
buildingPermits['POSTAL'] = buildingPermits['POSTAL'].astype(str)
buildingPermits['WARD_GRID'] = buildingPermits['WARD_GRID'].astype(str)
buildingPermits['DESCRIPTION'] = buildingPermits['DESCRIPTION'].astype(str)
buildingPermits['STATUS'] = buildingPermits['STATUS'].astype(str)


KeyError: 'ISSUED_DATE'

In [5]:
buildingPermits

,Permit_Num,Permit_Type,Structure_Type,Work,Street_Num,Street_Name,Street_Type,Street_Direction,Postal,Geo_Id,Ward_Grid,Application_Date,Issued_Date,Completed_Date,Status,Description,Est_Const_Cost
0,00 119701 CMB,Residential Building Permit,SFD - Detached,Addition to Existing Building,11,BETHNAL,AVE,,M8Y,987568.0,W0508,2000-01-10,2000-04-19,2016-12-30,Closed,Addition to Single Family Dwelling. HVAC &...,38000
1,00 123349 CMB,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,326,KENNEDY,AVE,,M6P,7963538.0,W1307,2000-01-25,2000-01-25,2016-05-19,Closed,Install Gas Fire Place in existing masonry fre...,3000
2,00 125006 CMB,Residential Building Permit,SFD - Semi-Detached,New Building,27,WILKET CREEK,RD,,M3C,14130214.0,N2606,2000-02-02,2000-03-22,2016-12-06,Closed,CONSTRUCT TWO AND A HALF STORIES SEMI-DETACHED...,145000
3,00 125292 CMB,Non-Residential Building Permit,Group A & B,Alteration to Existing Building,323,KEELE,ST,,M6P,7583849.0,S1401,2000-02-02,2000-08-02,2016-04-15,Closed,alter plans as approved under permit # 424961 ...,10000
4,00 126024 CMB,Non-Residential Building Permit,Restaurant Greater Than 30 Seats,Alteration to Existing Building,200,BLOOR,ST,E,M4W,8536223.0,S2704,2000-02-07,2000-05-05,2016-07-25,Closed,make interior alterations to portion of 2nd fl...,270000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75850,99 251937 BLD,Small Residential Projects,SFD - Detached,Interior Alterations,7,BROOKE,AVE,,M5M,493031.0,N0824,1999-07-30,1999-11-16,2018-09-14,Closed,REV TO B9901236,0
75851,99 251943 BLD,Small Residential Projects,SFD - Detached,Interior Alterations,452,GLENGARRY,AVE,,M5M,521259.0,N0824,1999-09-07,1999-11-16,2018-09-17,Closed,REV TO B9902003,0
75852,99 251946 BLD,Small Residential Projects,SFD - Detached,Interior Alterations,375,OLD ORCHARD,GRV,,M5M,546964.0,N0824,1999-09-10,1999-10-19,2018-09-17,Closed,REV TO B9901834,0
75853,99 251948 BLD,Small Residential Projects,SFD - Detached,Interior Alterations,522,FAIRLAWN,AVE,,M5M,513957.0,N0824,1999-09-14,1999-12-13,2018-09-14,Closed,REV:B9902368,0


In [6]:
demolition

,IBMS Address,Address of Existing Rental Building,RH File Number,City Council Approval Date,Type,Total Rental Homes for Demolition,Affordable Rental Homes for Demolition,Mid-Range Rental Homes for Demolition,High-End Rental Homes for Demolition,Total Rental Homes Replaced,Affordable Rental Homes Replaced,Mid-Range Rental Homes Replaced,High-End Rental Homes Replaced
0,75 MUTUAL ST,"75, 77, 83 MUTUAL ST",14 183555 STE 27 RH,2017-01-31,Demolition - 6 Rental Units or More,22.0,NaN,NaN,NaN,22.0,21.0,1.0,NaN
1,1996 BATHURST ST,"1996, 1998 and 2000 Bathurst Street",15 270570 STE 21 RH,2017-03-09,Demolition - 6 Rental Units or More,62.0,61.0,1.0,0.0,62.0,61.0,1.0,NaN
2,125 PARLIAMENT ST,125 Parliament Street and 50 Power Street,13 181698 STE 28 RH,2017-05-24,Demolition - 6 Rental Units or More,15.0,12.0,3.0,0.0,15.0,12.0,3.0,NaN
3,480 YONGE ST,"3 Grosvenor Street (492-494 Yonge Street), 484...",14 267359 STE 27 RH,2017-05-24,Demolition - 6 Rental Units or More,26.0,15.0,6.0,5.0,26.0,15.0,6.0,5.0
4,30 ERSKINE AVE,30 Erskine Avenue,13 116882 NNY 25 RH,2017-07-04,Demolition - 6 Rental Units or More,6.0,0.0,6.0,0.0,6.0,NaN,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Johanna Hashim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,"Senior Planner, Housing Policy",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,"City Planning, Strategic Initiatives, Policy &...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,Johanna.Hashim@toronto.ca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# generating surrogate keys
buildingPermits['Permit_Key'] = buildingPermits.index + 1

# Move 'Permit_Key' to the first position
cols = ['Permit_Key'] + [col for col in buildingPermits.columns if col != 'Permit_Key']

# Reorder the DataFrame
buildingPermits = buildingPermits[cols]

In [11]:
buildingPermits[:10]

,Permit_Key,Permit_Num,Permit_Type,Structure_Type,Work,Street_Num,Street_Name,Street_Type,Street_Direction,Postal,Geo_Id,Ward_Grid,Application_Date,Issued_Date,Completed_Date,Status,Description,Est_Const_Cost
0,1,00 119701 CMB,Residential Building Permit,SFD - Detached,Addition to Existing Building,11,BETHNAL,AVE,,M8Y,987568.0,W0508,2000-01-10,2000-04-19,2016-12-30,Closed,Addition to Single Family Dwelling. HVAC &...,38000
1,2,00 123349 CMB,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,326,KENNEDY,AVE,,M6P,7963538.0,W1307,2000-01-25,2000-01-25,2016-05-19,Closed,Install Gas Fire Place in existing masonry fre...,3000
2,3,00 125006 CMB,Residential Building Permit,SFD - Semi-Detached,New Building,27,WILKET CREEK,RD,,M3C,14130214.0,N2606,2000-02-02,2000-03-22,2016-12-06,Closed,CONSTRUCT TWO AND A HALF STORIES SEMI-DETACHED...,145000
3,4,00 125292 CMB,Non-Residential Building Permit,Group A & B,Alteration to Existing Building,323,KEELE,ST,,M6P,7583849.0,S1401,2000-02-02,2000-08-02,2016-04-15,Closed,alter plans as approved under permit # 424961 ...,10000
4,5,00 126024 CMB,Non-Residential Building Permit,Restaurant Greater Than 30 Seats,Alteration to Existing Building,200,BLOOR,ST,E,M4W,8536223.0,S2704,2000-02-07,2000-05-05,2016-07-25,Closed,make interior alterations to portion of 2nd fl...,270000
5,6,00 126923 CMB,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,322,INGLEWOOD,DR,,M4T,824997.0,S2701,2000-02-10,2000-03-24,2016-03-17,Closed,build 1st and 2nd floor rear addition (125.53...,120000
6,7,00 133738 CMB,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,304,HEATH,ST,E,M4T,818899.0,S2701,2000-03-09,2000-05-09,2016-11-23,Closed,"construct 1 storey addition to front, construc...",50000
7,8,00 137494 CMB,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,42,GLENADEN,AVE,E,M8Y,7862983.0,W0508,2000-03-23,2000-08-10,2016-02-10,Closed,Addition/basement walkout etc. to single famil...,70000
8,9,00 138962 CMB,Residential Building Permit,SFD - Detached,Addition to Existing Building,250,PRINCE EDWARD,DR,S,,6942825.0,W0509,2000-03-28,2000-06-27,2016-09-23,Closed,Addition to siingle family dwelling. HVAC dra...,24000
9,10,00 138974 CMB,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,101,CONSTANCE,ST,,M6R,786674.0,S1402,2000-03-28,2000-03-28,2016-07-26,Closed,One storey addition to rear of existing single...,10000


In [12]:
# generating surrogate keys
demolition['Demolition_Key'] = demolition.index + 1

# Move 'Permit_Key' to the first position
cols = ['Demolition_Key'] + [col for col in demolition.columns if col != 'Demolition_Key']

# Reorder the DataFrame
demolition = demolition[cols]

In [13]:
demolition[:10]

,Demolition_Key,IBMS Address,Address of Existing Rental Building,RH File Number,City Council Approval Date,Type,Total Rental Homes for Demolition,Affordable Rental Homes for Demolition,Mid-Range Rental Homes for Demolition,High-End Rental Homes for Demolition,Total Rental Homes Replaced,Affordable Rental Homes Replaced,Mid-Range Rental Homes Replaced,High-End Rental Homes Replaced
0,1,75 MUTUAL ST,"75, 77, 83 MUTUAL ST",14 183555 STE 27 RH,2017-01-31,Demolition - 6 Rental Units or More,22.0,NaN,NaN,NaN,22.0,21.0,1.0,NaN
1,2,1996 BATHURST ST,"1996, 1998 and 2000 Bathurst Street",15 270570 STE 21 RH,2017-03-09,Demolition - 6 Rental Units or More,62.0,61.0,1.0,0.0,62.0,61.0,1.0,NaN
2,3,125 PARLIAMENT ST,125 Parliament Street and 50 Power Street,13 181698 STE 28 RH,2017-05-24,Demolition - 6 Rental Units or More,15.0,12.0,3.0,0.0,15.0,12.0,3.0,NaN
3,4,480 YONGE ST,"3 Grosvenor Street (492-494 Yonge Street), 484...",14 267359 STE 27 RH,2017-05-24,Demolition - 6 Rental Units or More,26.0,15.0,6.0,5.0,26.0,15.0,6.0,5.0
4,5,30 ERSKINE AVE,30 Erskine Avenue,13 116882 NNY 25 RH,2017-07-04,Demolition - 6 Rental Units or More,6.0,0.0,6.0,0.0,6.0,NaN,6.0,NaN
5,6,59 MUTUAL ST,"59, 63, 65, 67, 69 and 71 Mutual Street",16 101809 STE 27 RH,2017-07-04,Demolition - 6 Rental Units or More,12.0,7.0,4.0,1.0,12.0,7.0,4.0,1.0
6,7,703 SOUDAN AVE,"1674 Bayview Avenue, 703 and 707 Soudan Avenue...",14 227490 STE 22 RH,2017-07-04,Demolition - 6 Rental Units or More,38.0,14.0,21.0,3.0,38.0,14.0,21.0,3.0
7,8,40 MOCCASIN TRL,"50 and 60 Green Belt Drive, and 40 Moccasin Trail",13 173075 NNY 34 RH,2017-07-04,Demolition - 6 Rental Units or More,83.0,59.0,24.0,0.0,67.0,43.0,24.0,NaN
8,9,2525 BATHURST ST,2525 Bathurst Street,16 152046 NNY 16 RH,2017-10-02,Demolition - 6 Rental Units or More,33.0,6.0,26.0,1.0,33.0,6.0,26.0,1.0
9,10,650 KINGSTON RD,650-652 Kingston Road and 2 Main Street,15 215445 STE 32 RH,2017-11-07,Demolition - 6 Rental Units or More,11.0,7.0,4.0,0.0,11.0,7.0,4.0,NaN


In [15]:
# Building the date dimension

earliest_date = buildingPermits['Completed_Date'].min()
latest_date = buildingPermits['Completed_Date'].max()

print(f"Earliest Completed Date: {earliest_date}")
print(f"Latest Completed Date: {latest_date}")

TypeError: '<=' not supported between instances of 'str' and 'float'

In [16]:
# Print a Series with the data type of each column
print(buildingPermits.dtypes)

Permit_Key            int64
Permit_Num           object
Permit_Type          object
Structure_Type       object
Work                 object
Street_Num           object
Street_Name          object
Street_Type          object
Street_Direction     object
Postal               object
Geo_Id              float64
Ward_Grid            object
Application_Date     object
Issued_Date          object
Completed_Date       object
Status               object
Description          object
Est_Const_Cost        int64
dtype: object
